# Combine All Model Results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/Tilburg DSS/Thesis/Data

In [ ]:
import pandas as pd

In [ ]:
df_VADER_Textblob=pd.read_pickle("VADER_Textblob.pkl")

In [ ]:
df_Roberta_Latest = pd.read_pickle("Roberta_Latest.pkl")

In [ ]:
df_Roberta_Latest=pd.DataFrame(df_Roberta_Latest, columns=['id', 'date','Dates', 'Time', 'tweet', 'hashtags', 'Roberta_Latest'])

In [ ]:
df_BERTweet = pd.read_pickle("BERTweet.pkl")

In [ ]:
df_BERTweet=pd.DataFrame(df_BERTweet, columns=['id', 'date','Dates', 'Time', 'tweet', 'hashtags', 'BERTweet'])

In [ ]:
from functools import reduce  

In [ ]:
#COMBINE ALL USED MODELS
data_merge1 = reduce(lambda left, right:     # Merge pandas DataFrames
                     pd.merge(left , right,
                              on = ['id', 'date','Dates', 'Time', 'tweet']),
                     [df_VADER_Textblob, df_BERTweet,  df_Roberta_Latest])
Models=pd.DataFrame(data_merge1)                   # Print merged DataFrame


In [ ]:
Models=Models.drop(columns=["hashtags_x"])

In [ ]:
Models= Models[['id', 'date', 'tweet', 'Dates', 'Time','hashtags_y','TextBlob_Polarity_lem', 'TextBlob_Polarity_unproc',
       'polarity_lem','unprocessed_VADER', 'BERTweet','Roberta_Latest']]

## Convert the labels as Positive=1, Negative=-1, Neutral=0

In [ ]:
preds=[]
for i in Models.BERTweet:
    for j in i:
        if j["label"].startswith('P'):
            preds.append(1)
        elif j["label"].startswith('NEU'):
            preds.append(0)
        else:
            preds.append(-1)

In [ ]:
Models["BERTweet"]=pd.DataFrame(data=preds)

In [ ]:
preds2=[]
for i in Models.Roberta_Latest:
    for j in i:
        if j["label"].startswith('p'):
            preds2.append(1)
        elif j["label"].startswith('neu'):
            preds2.append(0)
        else:
            preds2.append(-1)

In [ ]:
Models["Roberta_Latest"]=pd.DataFrame(data=preds2)

In [ ]:
#Convert the scores 
def getScores(score):
      if score < 0:
        return -1
      elif score == 0:
        return 0
      else:
        return 1


In [ ]:
Models["TextBlob_Polarity_lem"] = Models["TextBlob_Polarity_lem"].apply(lambda x: getScores(x))

In [ ]:
Models['TextBlob_Polarity_unproc'] = Models['TextBlob_Polarity_unproc'].apply(lambda x: getScores(x))

In [ ]:
def getScores2(score):
      if score >= 0.35 :
        return 1
      elif score <= -0.35:
        return -1
      else:
        return 0

In [ ]:
Models["polarity_lem"] = Models["polarity_lem"].apply(lambda x: getScores2(x))

In [ ]:
Models["unprocessed_VADER"] = Models["unprocessed_VADER"].apply(lambda x: getScores2(x))

In [ ]:
Models.to_pickle("Models_with_Labels.pkl")

In [ ]:
Models.to_excel("Models_with_Labels.xlsx")